In [36]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
import itertools
import random
import matplotlib.pyplot as plt
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import AutoEmbeddingComposite
from dwave.embedding.chain_strength import uniform_torque_compensation
import dimod
import timeit
import time

In [37]:
#繰り返さない！
dw_sampler = DWaveSampler(
        endpoint="https://cloud.dwavesys.com/sapi",
        solver = 'DW_2000Q_6',
        token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09",
        retry_interval=5
    )
# インスタンス作成
qa_sampler = AutoEmbeddingComposite(dw_sampler)

In [38]:
def calc_marginals(df):                   
    return np.array([                      
        sum(df['Y']), #0
        np.dot(df['Y'], df['SEX']),      #1
        np.dot(df['Y'], df['AOP']),    #2  
    ])     

In [39]:
def make_Hamiltonian(df):
    t_list = calc_marginals(df)
    N=len(df)
    dup_list = [(i, i) for i in range(N)]
    comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
    
    lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
    quad_Y = [2 for (i, j) in comb_list] #異なるy同士
    num_Y = t_list[0]**2 #数字の二乗
    
    SEX = df['SEX'].iloc
    lin_SEX  = [(SEX[i] - 2 * t_list[1]) * SEX[i] for (i, _) in dup_list]
    quad_SEX  = [2*SEX[i] * SEX[j] for (i, j) in comb_list]
    num_SEX  = t_list[1]**2
    
    AOP = df['AOP'].iloc
    lin_AOP = [(AOP[i] - 2 * t_list[2]) * AOP[i] for (i, _) in dup_list]
    quad_AOP = [2*AOP[i] * AOP[j] for (i, j) in comb_list]
    num_AOP = t_list[2]**2
    
    #lin
    lin_list = [sum(lin) for lin in zip(lin_Y, lin_SEX, lin_AOP)]
    lin = {i: lin_list[i] for (i, _) in dup_list}
    
    #quad
    quad_values = [sum(quad) for quad in zip(quad_Y, quad_SEX, quad_AOP)]
    quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}
    
    #num
    num = num_Y + num_SEX + num_AOP
    
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

In [40]:
def find_valid_y(df, chain_strength, num_reads):                                                        
    #bqm
    bqm = make_Hamiltonian(df)

    #method
    res = qa_sampler.sample(bqm, chain_strength = chain_strength, chain_break_fraction=True, num_reads=num_reads)
    
    df_list_list = []
    cols = ['sample', 'energy', 'num_occurrences', 'chain_break_fraction']
    for sample, energy, num_occurrences, chain_break_fraction in res.data(['sample', 'energy', 'num_occurrences', 'chain_break_fraction']):
        df_list_list.append([list(sample.values()), energy, num_occurrences, chain_break_fraction])
        
    
    valid_y_info_list = []
    valid_y_list = []
    valid_y_num = 0    
    for df_list in df_list_list:
        if df_list[1] == 0.0:
            valid_y_num += 1                                        
            valid_y_info_list.append(df_list)
            valid_y_list.append(df_list[0])
                                                      
    return  valid_y_info_list, valid_y_num, valid_y_list

In [ ]:
def find_valid_y_ntimes(df, chain_strength, num_reads, n):                                                        
    #bqm
    bqm = make_Hamiltonian(df)

    df_list_list = []
    cols = ['sample', 'energy', 'num_occurrences', 'chain_break_fraction']
    
    for _ in range(n):
        res = qa_sampler.sample(bqm, chain_strength = chain_strength, chain_break_fraction=True, num_reads=num_reads)
        for sample, energy, num_occurrences, chain_break_fraction in res.data(['sample', 'energy', 'num_occurrences', 'chain_break_fraction']):
            df_list_list.append([list(sample.values()), energy, num_occurrences, chain_break_fraction])
        
    
    valid_y_info_list = []
    valid_y_list = []
    valid_y_num = 0    
    for df_list in df_list_list:
        if df_list[1] == 0.0:
            valid_y_num += 1                                        
            valid_y_info_list.append(df_list)
            valid_y_list.append(df_list[0])
                                                      
    return  valid_y_info_list, valid_y_num, valid_y_list

In [41]:
def y_num_t1_hist(valid_y_info_df, save_plot_path):
    LI = list(df['LI'])
    hist_dic = {}
    
    for valid_y in valid_y_info_df['sample']:
        t1 = int(np.dot(LI, valid_y))
        if t1 in hist_dic.keys():
            hist_dic[t1] += 1
        else:
            hist_dic[t1] = 1
            
    plt.xlabel('value of t1')
    plt.ylabel('number of samples')
    plt.bar(hist_dic.keys(), hist_dic.values())
    plt.savefig(path)
    plt.show()
    
    return hist_dic

In [ ]:
def make_result_df_(valid_y_info_list, save_path):
    cols = ['sample', 'energy', 'num_occurrences', 'chain_break_fraction']
    valid_y_info_df = pd.DataFrame(index=[], columns=cols)

    for valid_y_info in valid_y_info_list:
        if all(valid_y_info[0] != p for p in valid_y_info_df['sample'].values.tolist()):
            record = pd.Series(valid_y_info, index=valid_y_info_df.columns)
            valid_y_info_df = valid_y_info_df.append(record, ignore_index=True)
    
    valid_y_info_df.to_csv(save_path)
    return valid_y_info_df

In [ ]:
def make_result_df(n, chain_strength, num_reads, save_path):
    cols = ['sample', 'energy', 'num_occurrences', 'chain_break_fraction']
    valid_y_info_df = pd.DataFrame(index=[], columns=cols)

    for _ in range(n):
        valid_y_info_list, valid_y_num, valid_y_list =find_valid_y(df, chain_strength=chain_strength, num_reads=num_reads)
        for valid_y_info in valid_y_info_list:
            if all(valid_y_info[0] != p for p in valid_y_info_df['sample'].values.tolist()):
                record = pd.Series(valid_y_info, index=valid_y_info_df.columns)
                valid_y_info_df = valid_y_info_df.append(record, ignore_index=True)
    
    valid_y_info_df.to_csv(save_path)
    return valid_y_info_df

In [42]:
def just_know_number(n, chain_strength, num_reads):
    valid_y_list = []

    for _ in range(n):
        valid_y_info_list, valid_y_num =find_valid_y(df, chain_strength=chain_strength, num_reads=num_reads)
        for valid_y_info in valid_y_info_list:
            if all(valid_y_info[0] != p for p in valid_y_list):
                valid_y_list.append(valid_y_info[0])
                
    return len(valid_y_list)

## trial
- 20bit

In [43]:
!pwd

/Users/shihosato/src/github.com/twinkle13531/master_degree/202011/scripts/functions


In [44]:
df = pd.read_csv('../../input/ost20.csv', sep=',', index_col=0)
print('20 bit datasets t1:', np.dot(df['Y'], df['LI']))

20 bit datasets t1: 9


### trial last

In [ ]:
valid_y_info_list, valid_y_num, valid_y_list = find_valid_y(df, chain_strength, num_reads)

### chain_strength = 10

In [9]:
n = 2
chain_strength = 10
num_reads = 1000
save_path = '../../output/QA/DwSamplerAutoEmbed_n2_ChainStrength10_NumReads1000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [10]:
valid_y_info_df 

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.00
1,"[1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, ...",0.0,1,0.00
2,"[0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, ...",0.0,1,0.00
3,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, ...",0.0,3,0.00
4,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, ...",0.0,1,0.00
5,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...",0.0,1,0.00
6,"[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.00
7,"[0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, ...",0.0,1,0.00
8,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, ...",0.0,1,0.00
9,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, ...",0.0,1,0.00


In [11]:
n = 3
chain_strength = 10
num_reads = 1000
save_path = '../../output/QA/DwSamplerAutoEmbed_n3_ChainStrength10_NumReads1000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [12]:
valid_y_info_df 

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, ...",0.0,1,0.0
1,"[1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.0
2,"[1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, ...",0.0,1,0.0
3,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, ...",0.0,1,0.0
4,"[1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
124,"[1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
125,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, ...",0.0,2,0.0
126,"[1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
127,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0


In [13]:
n = 10
chain_strength = 10
num_reads = 1000
save_path = '../../output/QA/DwSamplerAutoEmbed_n10_ChainStrength10_NumReads1000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [14]:
valid_y_info_df 

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.0
1,"[1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, ...",0.0,1,0.0
2,"[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.0
3,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...",0.0,1,0.0
4,"[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
272,"[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, ...",0.0,2,0.0
273,"[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
274,"[1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.0
275,"[1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...",0.0,1,0.0


In [15]:
n = 100
chain_strength = 10
num_reads = 100
save_path = '../../output/QA/DwSamplerAutoEmbed_n100_ChainStrength10_NumReads100.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [16]:
valid_y_info_df 

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.00
1,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, ...",0.0,1,0.00
2,"[1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.0,1,0.00
3,"[1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, ...",0.0,1,0.00
4,"[1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.0,1,0.00
...,...,...,...,...
406,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.05
407,"[1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, ...",0.0,2,0.05
408,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",0.0,1,0.00
409,"[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, ...",0.0,1,0.00


In [17]:
n = 1
chain_strength = 10
num_reads = 10000
save_path = '../../output/QA/DwSamplerAutoEmbed_n1_ChainStrength10_NumReads10000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [18]:
valid_y_info_df 

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, ...",0.0,1,0.00
1,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.10
2,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, ...",0.0,1,0.05
3,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.05
4,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, ...",0.0,1,0.15
...,...,...,...,...
162,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, ...",0.0,1,0.00
163,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, ...",0.0,2,0.00
164,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.00
165,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...",0.0,2,0.00


In [19]:
n = 1000
chain_strength = 10
num_reads = 10
save_path = '../../output/QA/DwSamplerAutoEmbed_n1000_ChainStrength10_NumReads10.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [20]:
valid_y_info_df 

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, ...",0.0,1,0.00
1,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, ...",0.0,1,0.00
2,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, ...",0.0,1,0.10
3,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, ...",0.0,1,0.05
4,"[1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.05
...,...,...,...,...
439,"[1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, ...",0.0,1,0.05
440,"[1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, ...",0.0,1,0.00
441,"[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.00
442,"[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.00


In [21]:
#error
n = 10000
chain_strength = 10
num_reads = 1
save_path = '../../output/QA/DwSamplerAutoEmbed_n10000_ChainStrength10_NumReads1.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

### chain_strength = 9

In [22]:
#oserror
n = 1
chain_strength = 9
num_reads = 10000
save_path = '../../output/QA/DwSamplerAutoEmbed_n1_ChainStrength9_NumReads10000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [23]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.0,68,0.05
1,"[1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, ...",0.0,1,0.25
2,"[1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, ...",0.0,1,0.20
3,"[1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, ...",0.0,1,0.25
4,"[1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.25
...,...,...,...,...
153,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.00
154,"[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.00
155,"[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.00
156,"[0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, ...",0.0,1,0.00


In [24]:
n = 10
chain_strength = 9
num_reads = 1000
save_path = '../../output/QA/DwSamplerAutoEmbed_n10_ChainStrength9_NumReads1000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [25]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, ...",0.0,1,0.1
1,"[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, ...",0.0,2,0.0
2,"[1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, ...",0.0,2,0.0
3,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
4,"[1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
246,"[1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.0
247,"[0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, ...",0.0,1,0.0
248,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, ...",0.0,1,0.0
249,"[1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0


In [26]:
n = 100
chain_strength = 9
num_reads = 100
save_path = '../../output/QA/DwSamplerAutoEmbed_n100_ChainStrength9_NumReads100.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [27]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.00
1,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.10
2,"[0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.10
3,"[0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.05
4,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.00
...,...,...,...,...
265,"[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.00
266,"[1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, ...",0.0,1,0.00
267,"[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, ...",0.0,2,0.05
268,"[0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.00


In [28]:
n = 1000
chain_strength = 9
num_reads = 10
save_path = '../../output/QA/DwSamplerAutoEmbed_n1000_ChainStrength9_NumReads10.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [29]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.0,1,0.05
1,"[0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.15
2,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.05
3,"[1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...",0.0,2,0.05
4,"[1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.10
...,...,...,...,...
341,"[0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.05
342,"[0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.05
343,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",0.0,1,0.00
344,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.15


In [30]:
n = 10000
chain_strength = 9
num_reads = 1
save_path = '../../output/QA/DwSamplerAutoEmbed_n10000_ChainStrength9_NumReads1.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [31]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, ...",0.0,1,0.00
1,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.00
2,"[1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.00
3,"[1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, ...",0.0,1,0.00
4,"[0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, ...",0.0,1,0.10
...,...,...,...,...
325,"[1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, ...",0.0,1,0.10
326,"[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.05
327,"[1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, ...",0.0,1,0.20
328,"[1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, ...",0.0,1,0.00


### chain_strength = 11

In [32]:
n = 1
chain_strength = 11
num_reads = 10000
save_path = '../../output/QA/DwSamplerAutoEmbed_n1_ChainStrength11_NumReads10000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [33]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.0
1,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, ...",0.0,2,0.0
2,"[1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, ...",0.0,1,0.0
3,"[1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.0
4,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, ...",0.0,1,0.0
...,...,...,...,...
269,"[1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.0
270,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, ...",0.0,1,0.0
271,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, ...",0.0,1,0.0
272,"[0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, ...",0.0,1,0.0


In [34]:
n = 10
chain_strength = 11
num_reads = 1000
save_path = '../../output/QA/DwSamplerAutoEmbed_n10_ChainStrength11_NumReads1000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [35]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.0
1,"[1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",0.0,1,0.0
2,"[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0
3,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.0
4,"[1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
286,"[1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.0
287,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0
288,"[1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0
289,"[1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.0


In [36]:
n = 100
chain_strength = 11
num_reads = 100
save_path = '../../output/QA/DwSamplerAutoEmbed_n100_ChainStrength11_NumReads100.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [37]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.0,1,0.0
1,"[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, ...",0.0,1,0.0
2,"[1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, ...",0.0,1,0.0
3,"[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, ...",0.0,1,0.0
4,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, ...",0.0,1,0.0
...,...,...,...,...
485,"[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, ...",0.0,1,0.0
486,"[1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, ...",0.0,1,0.0
487,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, ...",0.0,1,0.0
488,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, ...",0.0,1,0.0


In [38]:
n = 1000
chain_strength = 11
num_reads = 10
save_path = '../../output/QA/DwSamplerAutoEmbed_n1000_ChainStrength11_NumReads10.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [39]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, ...",0.0,1,0.00
1,"[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, ...",0.0,1,0.00
2,"[1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, ...",0.0,1,0.00
3,"[0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.00
4,"[0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.05
...,...,...,...,...
461,"[1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, ...",0.0,1,0.00
462,"[1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, ...",0.0,1,0.00
463,"[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.05
464,"[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, ...",0.0,1,0.00


In [40]:
n = 10000
chain_strength = 11
num_reads = 1
save_path = '../../output/QA/DwSamplerAutoEmbed_n10000_ChainStrength11_NumReads1.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

OSError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
valid_y_info_df

### chain_strength = 12

In [17]:
n = 1
chain_strength = 12
num_reads = 10000
save_path = '../../output/QA/DwSamplerAutoEmbed_n1_ChainStrength12_NumReads10000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

OSError: 

In [ ]:
valid_y_info_df

In [9]:
n = 10
chain_strength = 12
num_reads = 1000
save_path = '../../output/QA/DwSamplerAutoEmbed_n10_ChainStrength12_NumReads1000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [10]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, ...",0.0,1,0.0
1,"[1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, ...",0.0,3,0.0
2,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, ...",0.0,1,0.0
3,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...",0.0,1,0.0
4,"[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.0
...,...,...,...,...
531,"[1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, ...",0.0,1,0.0
532,"[0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.0
533,"[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ...",0.0,1,0.0
534,"[1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, ...",0.0,1,0.0


In [11]:
n = 100
chain_strength = 12
num_reads = 100
save_path = '../../output/QA/DwSamplerAutoEmbed_n100_ChainStrength12_NumReads100.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [12]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
1,"[1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.0
2,"[1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, ...",0.0,1,0.0
3,"[0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, ...",0.0,1,0.0
4,"[1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
441,"[1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, ...",0.0,1,0.0
442,"[0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, ...",0.0,1,0.0
443,"[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, ...",0.0,1,0.0
444,"[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0


In [13]:
n = 1000
chain_strength = 12
num_reads = 10
save_path = '../../output/QA/DwSamplerAutoEmbed_n1000_ChainStrength12_NumReads10.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [14]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, ...",0.0,1,0.0
1,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.0
2,"[1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.0
3,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.0
4,"[1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.0
...,...,...,...,...
445,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.0
446,"[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
447,"[1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, ...",0.0,1,0.0
448,"[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, ...",0.0,1,0.0


In [16]:
n = 10000
chain_strength = 12
num_reads = 1
save_path = '../../output/QA/DwSamplerAutoEmbed_n10000_ChainStrength12_NumReads1.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

OSError: 

In [ ]:
valid_y_info_df

### chain_strength = 13

In [10]:
n = 1
chain_strength = 13
num_reads = 10000
save_path = '../../output/QA/DwSamplerAutoEmbed_n1_ChainStrength13_NumReads10000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [11]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, ...",0.0,1,0.0
1,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.0
2,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, ...",0.0,1,0.0
3,"[1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, ...",0.0,1,0.0
4,"[0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
290,"[1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.0
291,"[1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, ...",0.0,1,0.0
292,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.0
293,"[1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, ...",0.0,1,0.0


In [12]:
n = 10
chain_strength = 13
num_reads = 1000
save_path = '../../output/QA/DwSamplerAutoEmbed_n10_ChainStrength13_NumReads1000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [13]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, ...",0.0,1,0.00
1,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, ...",0.0,1,0.05
2,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.00
3,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.00
4,"[1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.00
...,...,...,...,...
385,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, ...",0.0,1,0.00
386,"[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, ...",0.0,1,0.00
387,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.00
388,"[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.00


In [14]:
n = 100
chain_strength = 13
num_reads = 100
save_path = '../../output/QA/DwSamplerAutoEmbed_n100_ChainStrength13_NumReads100.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [15]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.0
1,"[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, ...",0.0,1,0.0
2,"[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0
3,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, ...",0.0,1,0.0
4,"[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
395,"[1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0
396,"[0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, ...",0.0,1,0.0
397,"[0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.0
398,"[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.0


In [16]:
n = 1000
chain_strength = 13
num_reads = 10
save_path = '../../output/QA/DwSamplerAutoEmbed_n1000_ChainStrength13_NumReads10.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [17]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.0
1,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, ...",0.0,1,0.0
2,"[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, ...",0.0,1,0.0
3,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.0
4,"[0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, ...",0.0,1,0.0
...,...,...,...,...
413,"[1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, ...",0.0,1,0.0
414,"[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.0
415,"[0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, ...",0.0,1,0.0
416,"[0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, ...",0.0,1,0.0


### chain_strength = 14

In [18]:
n = 100
chain_strength = 14
num_reads = 100
save_path = '../../output/QA/DwSamplerAutoEmbed_n100_ChainStrength14_NumReads100.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [19]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, ...",0.0,1,0.0
1,"[1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.0
2,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, ...",0.0,1,0.0
3,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, ...",0.0,1,0.0
4,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...",0.0,1,0.0
...,...,...,...,...
391,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.0
392,"[0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, ...",0.0,1,0.0
393,"[1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, ...",0.0,1,0.0
394,"[1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0


## 最後に確認

In [20]:
n = 10
chain_strength = 12
num_reads = 1000
save_path = '../../output/QA/DwSamplerAutoEmbed_n10_ChainStrength12_NumReads1000.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [21]:
valid_y_info_df#少ない...

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, ...",0.0,1,0.00
1,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...",0.0,1,0.05
2,"[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, ...",0.0,1,0.00
3,"[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.00
4,"[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, ...",0.0,1,0.00
...,...,...,...,...
364,"[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.00
365,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, ...",0.0,1,0.00
366,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.00
367,"[1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, ...",0.0,1,0.00


In [22]:
n = 100
chain_strength = 12
num_reads = 100
save_path = '../../output/QA/DwSamplerAutoEmbed_n100_ChainStrength12_NumReads100.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [23]:
valid_y_info_df#毎回毎回結果が変わるな

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.0
1,"[1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, ...",0.0,1,0.0
2,"[1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, ...",0.0,1,0.0
3,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",0.0,1,0.0
4,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
410,"[1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, ...",0.0,1,0.0
411,"[1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.0
412,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, ...",0.0,1,0.0
413,"[1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, ...",0.0,1,0.0


In [24]:
n = 1000
chain_strength = 12
num_reads = 10
save_path = '../../output/QA/DwSamplerAutoEmbed_n1000_ChainStrength12_NumReads10.csv'
valid_y_info_df = make_result_df(n, chain_strength, num_reads, save_path)

In [25]:
valid_y_info_df

,sample,energy,num_occurrences,chain_break_fraction
0,"[1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, ...",0.0,1,0.0
1,"[1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, ...",0.0,1,0.0
2,"[1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.0
3,"[0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, ...",0.0,1,0.0
4,"[1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, ...",0.0,1,0.0
...,...,...,...,...
459,"[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, ...",0.0,1,0.0
460,"[1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, ...",0.0,1,0.0
461,"[1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, ...",0.0,1,0.0
462,"[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, ...",0.0,1,0.0


In [28]:
n = 100
num_reads = 100
for chain_strength in [9, 10, 11, 12]:#topは11
    print(just_know_number(n, chain_strength, num_reads))

273
421
484
429


In [29]:
n = 10
num_reads = 1000
for chain_strength in [9, 10, 11, 12]::#topは10
    print(just_know_number(n, chain_strength, num_reads))

290
464
401
416


In [45]:
n = 1000
num_reads = 10
for chain_strength in [9, 10, 11, 12]:#topは11
    print(just_know_number(n, chain_strength, num_reads))

325
434
469
415
